# Operator Overloading: Doing It Right

- Operator overloading allows user-defined objects to interoperate with infix operators such as + and | or unary operators like - and ~.

## Operator Overloading 101

- We cannot overload operators for the built-in types.
- We cannot create new operators, only overload existing ones.
- A few operators can’t be overloaded: is, and, or, not (but the bitwise &, |, ~, can).

## Unary Operators

- - `__neg__` : Arithmetic unary negation
- + `__pos__` : Arithmetic unary plus
- ~ `__invert__` : Bitwise inverse of an integer: 
  - ~x == -(x+1). If x is 2 then ~x == -3.


It’s easy to support the unary operators. Simply implement the appropriate special method, which will receive just one argument: self. Use whatever logic makes sense in your class, but stick to the fundamental rule of operators: `always return a new object.`


```py
#  Example for Vector
def __neg__(self):
    return Vector(-x for x in self) 

def __pos__(self):
     return Vector(self)

```

## Overloading + for Vector Addition

```py
import itertools


def __add__(self, other):
    pairs = itertools.zip_longest(self, other, fillvalue=0.0)
    return Vector(a + b for a, b in pairs) 
```

To support operations involving objects of different types, Python implements a special dispatching mechanism for the infix operator special methods. 

1. If a has `__add__`, call a.`__add__`(b) and return result unless it’s NotImplemented
2. If a doesn’t have `__add__`, or calling it returns NotImplemented, check if b has `__radd__`, then call b.`__radd__`(a) and return result unless it’s NotImplemented.
3. If b doesn’t have `__radd__`, or calling it returns NotImplemented, raise TypeError with an unsupported operand types message.



Therefore, to make the mixed-type additions work, we need to implement the Vector.`__radd__` method.

```py
def __add__(self, other): # 
    pairs = itertools.zip_longest(self, other, fillvalue=0.0)
    return Vector(a + b for a, b in pairs)
 
 
def __radd__(self, other):
    return self + other
```


## Overloading * for Scalar Multiplication

```py
import numbers

def __mul__(self, scalar):
    if isinstance(scalar, numbers.Real):
        return Vector(n * scalar for n in self)
    else:
        return NotImplemented


def __rmul__(self, scalar):
    return self * scalar

```

## Rich Comparison Operators

- The same set of methods are used in forward and reverse operator calls. The rules are summarized in Table 13-2. For example, in the case of ==, both the forward and reverse calls invoke `__eq__`, only swapping arguments; and a forward call to `__gt__` is followed by a reverse call to `__lt__` with the swapped arguments.
- In the case of == and !=, if the reverse call fails, Python compares the object IDs instead of raising TypeError.

In [ ]:
class Vector:
    # many lines omitted
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and
                    all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented 


## Augmented Assignment Operators

If a class does not implement the in-place operators, the augmented
assignment operators are just syntactic sugar: a += b is evaluated exactly as a = a + b.

If you have `__add__` then `+= will work with no additional code`.

However, if you do implement an in-place operator method such as `__iadd__`, that method is called to compute the result of a += b. As the name says, those `operators are expected to change the left hand operand in place, and not create a new object as the result.`

# Chapter Summary

- restrictions Python imposes on operator overloading: no overloading of operators in built-in types, and overloading limited to existing operators, except for a few ones (is, and, or, not)
- unary and infix operators are supposed to produce results by creating new objects, and should never change their operands
- To support operations with other types, we return the NotImplemented special value—not an exception—allowing the interpreter to try again by swapping the operands and calling the reverse special method for that operator (e.g., `__radd__`)
- Mixing operand types means we need to detect when we get an operand we can’t handle. 
  - tried the operation, catching a TypeError exception if it happened
  - an explicit isinstance test
    -  explicit type checking is more predictable.
    -  avoid testing with a concrete class
-  The way Python evaluates  comparison operators  is slightly different, with a different logic for choosing the reverse method, and special fallback handling for == and !=, which never generate errors because Python compares the object IDs as a last resort
-  Python handles augmented assignment operators by default as a combination of plain operator followed by assignment,
-  For sequence types, + usually requires that both operands are of the same type, while += often accepts any iterable as the righthand operand.